In [1]:
%pip install boto3==1.21.2


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import required packages and establish session with default credentials

import boto3
import os
import io
import gzip
import shutil

# Set up the Boto3 session
session = boto3.Session()

# Check if an active token is present
sts_client = session.client("sts")
try:
    sts_client.get_caller_identity()
    print("Active token found!")
except boto3.exceptions.botocore.exceptions.NoCredentialsError:
    print("No active token found.")
    # Prompt for MFA code
    mfa_token = input("Enter MFA token code: ")

    # Assume role with MFA credentials
    sts_client = session.client("sts")
    response = sts_client.assume_role(
        RoleArn="arn:aws:iam::251357961920:role/AdminAccess",
        RoleSessionName="my-role-session",
        TokenCode=mfa_token,
    )

    # Set up a new Boto3 session with the assumed role credentials
    session = boto3.Session(
        aws_access_key_id=response["Credentials"]["AccessKeyId"],
        aws_secret_access_key=response["Credentials"]["SecretAccessKey"],
        aws_session_token=response["Credentials"]["SessionToken"],
    )
    print("Assumed role with MFA credentials!")


Active token found!


In [3]:
# Open S3 connection and set variables

s3 = session.client("s3")

# Replace 'bucket-name' and 'object-key' with the actual values
bucket_name = "commoncrawl"
object_key = "crawl-data/CC-MAIN-2022-05/wet.paths.gz"
project_path = "/Users/blakeenyart/programming/projects/sandbox/data-engineering-practice/Exercises/Exercise-3"

In [4]:
# Get the object from the bucket
content = s3.get_object(Bucket=bucket_name, Key=object_key)['Body'].read()

In [5]:
extracted_file_path = os.path.join(project_path, object_key.split('/')[-1][:-3])

next_object_key = ''

# Extract the .gz file to a new file
with gzip.GzipFile(fileobj=io.BytesIO(content)) as file:
    for line in file:
        clean_line = line.decode().strip()
        print(clean_line)
        next_object_key = clean_line
        break

crawl-data/CC-MAIN-2022-05/segments/1642320299852.23/wet/CC-MAIN-20220116093137-20220116123137-00000.warc.wet.gz


In [6]:
# Download much larger file indicated from the file path above
next_local_gz_path = os.path.join(project_path, next_object_key.split('/')[-1])

# Get the object from the bucket
s3.download_file(bucket_name, next_object_key, next_local_gz_path)

In [15]:
# Extract the .gz file to a new file
next_extracted_file_path = os.path.join(project_path, next_object_key.split('/')[-1][:-3])

with gzip.open(next_local_gz_path, "rb") as f_in:
    with open(next_extracted_file_path, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [21]:
# Stream the output of the file to the terminal
with open(next_extracted_file_path) as file:
    i = 0
    for line in file:
        print(line.strip())
        # Limit output to reduce local usage
        i += 1
        if i == 10:
            break

WARC/1.0
WARC-Type: warcinfo
WARC-Date: 2022-01-29T16:59:40Z
WARC-Filename: CC-MAIN-20220116093137-20220116123137-00000.warc.wet.gz
WARC-Record-ID: <urn:uuid:ffbe2340-34aa-43df-9b03-232751f3fbb8>
Content-Type: application/warc-fields
Content-Length: 372

Software-Info: ia-web-commons.1.1.10-SNAPSHOT-20220113011612
Extracted-Date: Sat, 29 Jan 2022 16:59:40 GMT


In [ ]:
# File clean-up afterwards
os.remove(next_extracted_file_path)
os.remove(next_local_gz_path)